In [ ]:
%matplotlib inline
from glob import glob 

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binned_statistic 
from batman import TransitModel

import sys
sys.path.insert(0, '../')
from linea import CheopsLightCurve, JointLightCurve, params_WASP189

p = params_WASP189()

In [ ]:
paths = glob('../linea/data/example_wasp189_visit*.fits')

lcs = JointLightCurve([CheopsLightCurve.from_fits(p) 
                       for p in paths])

In [ ]:
for lc in lcs: 
    lc.sigma_clip_flux(sigma_upper=3, sigma_lower=3, plot=True)
    plt.show()

In [ ]:
Xs = [lc.design_matrix() for lc in lcs]
X = lcs.combined_design_matrix(Xs)

all_lcs = lcs.concatenate()

eclipse_model = TransitModel(p, all_lcs.bjd_time[~all_lcs.mask], 
                             supersample_factor=3, 
                             transittype='secondary',
                             exp_time=all_lcs.bjd_time[1]-all_lcs.bjd_time[0]
                            ).light_curve(p) - 1

X = np.hstack([
    X, 
    eclipse_model[:, None]
])

In [ ]:
r = lcs.regress(X)

In [ ]:
print(f"Eclipse depth = {r.betas[-1] :.0f} ± {np.sqrt(np.diag(r.cov))[-1] :.0f} ppm")

In [ ]:
phases = np.concatenate([lc.phase(p)[~lc.mask] for lc in lcs])

sort = np.argsort(phases)

obs_eclipse = all_lcs.flux[~all_lcs.mask] / (X[:, :-1] @ r.betas[:-1])
eclipse_model = X[:, -1] * r.betas[-1]

bs = binned_statistic(phases[sort], obs_eclipse[sort], bins=30)
bincenters = 0.5 * (bs.bin_edges[1:] + bs.bin_edges[:-1])

fig, ax = plt.subplots(figsize=(8, 6), sharex=True)
ax.plot(phases, obs_eclipse, '.', color='silver')
ax.plot(phases[sort], eclipse_model[sort] + 1, 'r', lw=2)
ax.plot(bincenters, bs.statistic, 's', color='k')

for sp in ['right', 'top']:
    ax.spines[sp].set_visible(False)
ax.ticklabel_format(useOffset=False)

ax.set(xlabel='Phase', ylabel='Flux', 
       ylim=[0.99965, 1.0004]);